In [1]:
import numpy as np
import pandas as pd

In [3]:
# Define the KNN class
class KNN:
    def __init__(self, k=3, distance_metric='euclidean'):
        self.k = k
        self.distance_metric = distance_metric
        self.X_train=None
        self.y_train=None
        self.weighted = True


    def fit(self, X, y):
        # TODO: Implement the fit method
        self.X_train=X
        self.y_train=y

    def predict(self, X):
     # Convert X to numpy array
        probabilities = []
        # Compute distances between each point in X and all points in X_train
        distances = self.compute_distance(X, self.X_train)

        # Iterate over each sample's distances
        for dist in distances:
            k_indices = np.argsort(dist)[:self.k]
            k_nearest_labels = self.y_train[k_indices].astype(int)

            # Compute the probability of class 1
            prob_class_1 = np.sum(k_nearest_labels == 1) / self.k
            probabilities.append(prob_class_1)
        return np.array(probabilities)

    def compute_distance(self, X1, X2):
        # TODO: Implement distance computation based on self.distance_metric
        # Hint: Use numpy operations for efficient computation
        if self.distance_metric == 'euclidean':
            return np.sqrt(np.sum((X1[:, np.newaxis] - X2) ** 2, axis=2))
        elif self.distance_metric == 'manhattan':
            return np.sum(np.abs(X1[:, np.newaxis] - X2), axis=2)
        else:
            raise ValueError(f"Distance metric {self.distance_metric} not supported.")

In [5]:
def preprocess_data(train_path, test_path=None):
    # Load the train data
    train_data = pd.read_csv(train_path)
    test_data = pd.read_csv(test_path) if test_path else None

    # Concatenate train and test data if test data is provided
    if test_data is not None:
        combined_data = pd.concat([train_data, test_data], keys=['train', 'test'])
    else:
        combined_data = train_data

    # Drop unnecessary columns
    combined_data.drop(['CustomerId', 'Surname'], axis=1, inplace=True)

    # Handle missing values for numerical columns (replace with median)
    num_cols = combined_data.select_dtypes(include=[np.number]).columns.tolist()
    for col in num_cols:
        median_val = combined_data[col].median()
        combined_data[col].fillna(median_val, inplace=True)

    # Handle missing values for categorical columns (replace with mode)
    cat_cols = combined_data.select_dtypes(exclude=[np.number]).columns.tolist()
    for col in cat_cols:
        mode_val = combined_data[col].mode()[0]
        combined_data[col].fillna(mode_val, inplace=True)

    # One-hot encoding for 'Geography' (pandas get_dummies)
    combined_data = pd.get_dummies(combined_data, columns=['Geography'], drop_first=True)

    # Manual encoding for 'Gender' (replace 'Male' with 1 and 'Female' with 0)
    combined_data['Gender'] = combined_data['Gender'].replace({'Male': 1, 'Female': 0})

    # If test data is provided, split combined data back into train and test
    
    train_data = combined_data.xs('train')
    test_data = combined_data.xs('test')

        # Ensure 'Exited' column is dropped from test data (if present)
    if 'Exited' in test_data.columns:
        test_data.drop('Exited', axis=1, inplace=True)

        # Align test set with training set columns
    test_data = test_data.reindex(columns=train_data.drop('Exited', axis=1).columns, fill_value=0)

    # Separate features (X) and target (y) for training set
    X_train = train_data.drop('Exited', axis=1)
    y_train = train_data['Exited']

    # Feature scaling using manual robust scaling (subtract median and divide by IQR)
    X_train_scaled = X_train.copy()
    scaling_params = {}  # Dictionary to store the median and IQR for each column

    for col in X_train.columns:
        if X_train[col].dtype in [np.float64, np.int64]:  # Only scale numeric columns
            median_val = X_train[col].median()
            iqr_val = X_train[col].quantile(0.75) - X_train[col].quantile(0.25)
            
            # If IQR is 0 (constant feature), avoid division by 0 by scaling with 1
            if iqr_val == 0:
                iqr_val = 1
            
            # Store the scaling parameters
            scaling_params[col] = {'median': median_val, 'iqr': iqr_val}
            
            # Scale the training data
            X_train_scaled[col] = (X_train[col] - median_val) / iqr_val

    if test_data is not None:
        X_test_scaled = test_data.copy()
        for col in test_data.columns:
            if test_data[col].dtype in [np.float64, np.int64]:  # Only scale numeric columns
                # Use the median and IQR from the training set
                median_val = scaling_params[col]['median']
                iqr_val = scaling_params[col]['iqr']
                
                # Scale the test data using the training set's statistics
                X_test_scaled[col] = (test_data[col] - median_val) / iqr_val
        # Convert to numpy arrays and ensure proper 2D shape
        return np.array(X_train_scaled,dtype=np.float64), np.array(y_train,dtype=np.float64), np.array(X_test_scaled,dtype=np.float64)


In [7]:
def kfold(X, y, n_splits=5):
    # Shuffle the indices of X and y
    indices = np.arange(len(X))
    np.random.seed(42)
    np.random.shuffle(indices)

    # Split the indices into roughly equal parts
    fold_sizes = np.full(n_splits, len(X) // n_splits, dtype=int)
    fold_sizes[:len(X) % n_splits] += 1
    current = 0
    folds = []
    for fold_size in fold_sizes:
        start, stop = current, current + fold_size
        folds.append(indices[start:stop])
        current = stop

    return folds

def roc_auc(y_true, y_pred):
    # Convert to numpy arrays for easier manipulation
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)

    # Sort instances by the predicted score (y_pred)
    sorted_indices = np.argsort(y_pred)[::-1]
    y_true_sorted = y_true[sorted_indices]

    # Count the total number of positives and negatives
    pos_count = np.sum(y_true == 1)
    neg_count = np.sum(y_true == 0)

    # Calculate the true positive rate (TPR) and false positive rate (FPR)
    tpr = np.cumsum(y_true_sorted == 1) / pos_count
    fpr = np.cumsum(y_true_sorted == 0) / neg_count

    # Use the trapezoidal rule to calculate AUC
    auc = np.trapz(tpr, fpr)
    return auc

def cross_validate(X, y, knn, n_splits=5):
    
    # Initialize list to store AUC scores for each fold
    auc_scores = []

    # Perform manual KFold cross-validation
    folds = kfold(X, y, n_splits)

    # Perform cross-validation
    for fold in folds:
        # Split data into training and testing sets using index values
        test_index = fold
        train_index = np.setdiff1d(np.arange(len(X)), test_index)

        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        # Fit the KNN model on the training data
        knn.fit(X_train, y_train)

        # Predict the labels for the test set
        y_pred = knn.predict(X_test)

        # Calculate ROC AUC score based on binary classification
        auc = roc_auc(y_test, y_pred)
        auc_scores.append(auc)

    # Return the list of AUC scores and the average AUC
    return auc_scores, np.mean(auc_scores)


In [13]:
# Load and preprocess data
X, y, X_test = preprocess_data('./train.csv', './test.csv')
# Create and evaluate model
knn = KNN(k=5, distance_metric='euclidean')

# Perform cross-validation
cv_scores = cross_validate(X, y, knn)

print("Cross-validation scores:", cv_scores)

# TODO: hyperparamters tuning
# Hyperparameter tuning

def hyperparameter_tuning(X, y):
    # Define the grid of hyperparameters
    param_grid = {
        'k': [3, 5, 7, 9, 11,13,15,17,19,21,23,25,27,31,33,35,37],  # Test different values of k
        'distance_metric': ['euclidean', 'manhattan']  # Test different distance metrics
    }

    best_params = None
    best_score = 0

    # Grid search over the hyperparameters
    for k in param_grid['k']:
        for metric in param_grid['distance_metric']:
            knn = KNN(k=k, distance_metric=metric)
            scores, avg_auc = cross_validate(X, y, knn)
            print(f"Testing k={k}, metric={metric}, ROC={avg_auc}")

            # Keep track of the best hyperparameters
            if avg_auc > best_score:
                best_score = avg_auc
                best_params = {'k': k, 'distance_metric': metric}
    return best_params


# Get the best hyperparameters using cross-validation
best_params = hyperparameter_tuning(X, y)

# TODO: Train on full dataset with optimal hyperparameters and make predictions on test set
knn = KNN(k=best_params['k'], distance_metric=best_params['distance_metric'])
knn.fit(X, y)

# Make predictions on the test set
test_predictions = knn.predict(X_test)
# Save test predictions
pd.DataFrame({'id': pd.read_csv('./test.csv')['id'], 'Exited': test_predictions}).to_csv('submissions.csv', index=False)

/var/folders/td/3s3bcbys3qs05zfg3_jzxfhc0000gn/T/ipykernel_3878/3810826123.py:19: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  combined_data[col].fillna(median_val, inplace=True)
/var/folders/td/3s3bcbys3qs05zfg3_jzxfhc0000gn/T/ipykernel_3878/3810826123.py:25: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values a

Cross-validation scores: ([0.8575250265840324, 0.8824342640281919, 0.8674058425872627, 0.8601504131938914, 0.885742608249612], 0.8706516309285981)
Testing k=3, metric=euclidean, ROC=0.8443476329909609
Testing k=3, metric=manhattan, ROC=0.8401710968641847
Testing k=5, metric=euclidean, ROC=0.8706516309285981
Testing k=5, metric=manhattan, ROC=0.8677401821172422
Testing k=7, metric=euclidean, ROC=0.8845887436483502
Testing k=7, metric=manhattan, ROC=0.884359103919903
Testing k=9, metric=euclidean, ROC=0.891650819581761
Testing k=9, metric=manhattan, ROC=0.8898152650500535
Testing k=11, metric=euclidean, ROC=0.8971864256096829
Testing k=11, metric=manhattan, ROC=0.8957542164668038
Testing k=13, metric=euclidean, ROC=0.8996697530593858
Testing k=13, metric=manhattan, ROC=0.8983279137367746
Testing k=15, metric=euclidean, ROC=0.900885058925762
Testing k=15, metric=manhattan, ROC=0.9002097004876738
Testing k=17, metric=euclidean, ROC=0.9033155011550138
Testing k=17, metric=manhattan, ROC=0.9